In [3]:
import numpy as np
%matplotlib widget
import matplotlib.pyplot as plt
from opmd_viewer import OpenPMDTimeSeries

In [10]:
ts = OpenPMDTimeSeries('/tmp/nbody_gpu_test_run')
# ts = OpenPMDTimeSeries('/mnt/hdd/data/gpu_run_debug')
# ts = OpenPMDTimeSeries('/home/dominik/Code/openPMD/openPMD-example-datasets/example-3d/hdf5/')
ts

In [11]:
# Interactive GUI
ts.slider()

FigureCanvasNbAgg()

In [12]:
ts.h5_files

['/tmp/nbody_gpu_test_run/data00000000.h5',
 '/tmp/nbody_gpu_test_run/data00000100.h5',
 '/tmp/nbody_gpu_test_run/data00000200.h5',
 '/tmp/nbody_gpu_test_run/data00000300.h5',
 '/tmp/nbody_gpu_test_run/data00000400.h5',
 '/tmp/nbody_gpu_test_run/data00000500.h5',
 '/tmp/nbody_gpu_test_run/data00000600.h5',
 '/tmp/nbody_gpu_test_run/data00000700.h5',
 '/tmp/nbody_gpu_test_run/data00000800.h5',
 '/tmp/nbody_gpu_test_run/data00000900.h5',
 '/tmp/nbody_gpu_test_run/data00001000.h5']

In [13]:
import h5py
h5_files = [h5py.File(f) for f in ts.h5_files]

In [21]:
import pandas as pd
df = pd.read_json('/tmp/nbody_gpu_test_run/diagnostic_results.json').T
df["total_energy"] = df["kinetic_energy"] + df["potential_energy"]
df

,kinetic_energy,potential_energy,temperature,total_energy
0,2.754142e-18,-1.151071e-28,260.250204,2.754142e-18
100,2.754142e-18,-6.887468e-42,260.250204,2.754142e-18
200,2.754142e-18,-1.240557e-43,260.250204,2.754142e-18
300,2.754142e-18,-1.144423e-44,260.250204,2.754142e-18
400,2.754142e-18,-2.088754e-45,260.250204,2.754142e-18
500,2.754142e-18,-5.559591e-46,260.250204,2.754142e-18
600,2.754142e-18,-1.881001e-46,260.250204,2.754142e-18
700,2.754142e-18,-7.514263e-47,260.250204,2.754142e-18
800,2.754142e-18,-3.390975e-47,260.250204,2.754142e-18
900,2.754142e-18,-1.679855e-47,260.250204,2.754142e-18


In [26]:
df.drop('temperature', axis=1).plot()

FigureCanvasNbAgg()

In [14]:
print(dict(h5_files[0].attrs)['git_diff'])

diff --git a/config.json b/config.json
index b86da69..c643735 100644
--- a/config.json
+++ b/config.json
@@ -4,7 +4,7 @@
   "well_depth": 1.6540169269599997e-21
  },
  "N": 4096,
- "file_path": "/mnt/hdd/data/gpu_run_debug/data{0:08d}.h5",
+ "file_path": "/mnt/hdd/data/gpu_run_debug2/data{0:08d}.h5",
  "N_iterations": 100,
  "dt": 1e-09,
  "q": 0,
@@ -12,5 +12,5 @@
  "T": 273,
  "box_L": 1e-07,
  "save_every_x_iters": 10,
- "gpu": True
+ "gpu": true
 }
diff --git a/environment.yml b/environment.yml
index f72146a..dc97461 100644
--- a/environment.yml
+++ b/environment.yml
@@ -4,6 +4,7 @@ channels:
 dependencies:
   - python=3.6
   - numba # TODO pin versions
+  - astropy
   - cupy
   - numpy
   - pytest
diff --git a/nbody/initial_conditions.py b/nbody/initial_conditions.py
index 79a7ba1..bc752d4 100644
--- a/nbody/initial_conditions.py
+++ b/nbody/initial_conditions.py
@@ -8,7 +8,7 @@ import numpy as np
 import math
 import git
 
-k_B = 1.380_648_52e-23
+from nbody.constants import k_B
